In [ ]:
import os
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

# Nuevas importaciones para la sintaxis moderna (LCEL)
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [ ]:


# --- Carga de la API key (esto está perfecto) ---
try:
    with open("../gem_apikey.txt") as f:
        api_key = f.read().strip()
    print("API key cargada correctamente.")
except FileNotFoundError:
    print("Error: No se encontró el archivo '../gem_apikey.txt'.")
    api_key = None
# -----------------------------------------


def iniciar_chat():
    """
    Carga la base de datos vectorial y permite chatear con la personalidad del Dr. Deo
    utilizando la sintaxis moderna de LangChain (LCEL).
    """
    if not api_key:
        print("Proceso detenido. No se pudo cargar la API key.")
        return

    # --- CONFIGURACIÓN ---
    CARPETA_GUARDADO = "faiss_index_doc1"

    # 1. Carga de la base de datos vectorial
    print(f"Cargando la base de datos vectorial desde '{CARPETA_GUARDADO}'...")
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001", google_api_key=api_key
    )
    vectorstore = FAISS.load_local(
        CARPETA_GUARDADO, embeddings, allow_dangerous_deserialization=True
    )

    # Se define el retriever una sola vez
    retriever = vectorstore.as_retriever()

    # 2. Configuración del LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash", temperature=0.3, google_api_key=api_key
    )

    # 3.1 Prompt con personalidad y contexto
    system_tamplate = """Eres un asistente virtual que se comporta como un profesor de {knowledgeDomain} experto.
    Tu nombre es "IAsistente de {Teacher}". Eres amable, didáctico y te encanta {knowledgeDomain}. 
    REGLA ESTRICTA: Solo puedes responder preguntas relacionadas con {knowledgeDomain} basándote en el contexto proporcionado. Si la pregunta no está relacionada con estos temas, 
    responde: "Lo siento, mi especialidad es {knowledgeDomain}. No puedo responder preguntas sobre otros temas."""

    system_message_prompt = SystemMessagePromptTemplate.from_template(system_tamplate)

    # 3.2 Prompt para la pregunta del usuario
    human_template = "{pregunta_usuario}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    # 3.3 Combinación de los prompts en un ChatPromptTemplate

    chat_prompt_template = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    # 3.4 Definición de las variables del sistema
    chat_prompt_template = chat_prompt_template.partial(Teacher="Dr. Deo", knowledgeDomain="Medicina")

    # 4. Creación de la cadena de RAG completa con LCEL
    #    Esta es la parte principal de la corrección.
    cadena_rag = (
        {"contexto": retriever, "pregunta_usuario": RunnablePassthrough()}
        | chat_prompt_template
        | llm
        | StrOutputParser()
    )

    print("\n¡El Asistenteestá listo para responder preguntas!")
    print("Escribe 'salir' para terminar la conversación.")


    # 5. Bucle de chat
    while True:
        query = input("\nPregunta del estudiante: ")
        if query.lower() == "salir":
            break

        # Invocamos la cadena completa con la pregunta del usuario
        respuesta = cadena_rag.invoke(query)

        print("\nRespuesta del Asistente:")
        # La respuesta ya es un texto limpio gracias a StrOutputParser
        print(respuesta)


if __name__ == "__main__":
    iniciar_chat()

API key cargada correctamente.
Cargando la base de datos vectorial desde 'faiss_index_doc1'...

¡El Dr. Deo está listo para responder preguntas!
Escribe 'salir' para terminar la conversación.
